# MIDO

In [1]:
from mido import MidiFile
song = 'midi/Acordai-cello.mid'
mid = MidiFile(song)

In [2]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        if not msg.is_meta:
            print(msg)


Track 0: Violonchelo
control_change channel=0 control=121 value=0 time=0
control_change channel=0 control=100 value=0 time=0
control_change channel=0 control=101 value=0 time=0
control_change channel=0 control=6 value=12 time=0
control_change channel=0 control=100 value=127 time=0
control_change channel=0 control=101 value=127 time=0
program_change channel=0 program=42 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
note_on channel=0 note=63 velocity=80 time=0
control_change channel=0 control=2 value=80 time=0
control_change channel=1 control=121 value=0 time=0
control_change channel=1 control=100 value=0 time=0
control_change channel=1 control=101 value=0 time=0
control_change channel=1 control=6 value=12 time=0
control_change channel=1 control=100 value=127 time=0
control_change channel=1 control=101 value=127 time=0
prog

In [14]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import random
import os

def load_midi(path):
    return MidiFile(path)

def save_midi(mid, output_path):
    mid.save(output_path)

def faster_tempo(mid, factor=2):
    return _change_tempo(mid, 1 / factor)

def slower_tempo(mid, factor=2):
    return _change_tempo(mid, factor)

def accelerando_tempo(mid):
    return _gradual_tempo_change(mid, start_factor=1.0, end_factor=0.5)

def ritardando_tempo(mid):
    return _gradual_tempo_change(mid, start_factor=1.0, end_factor=1.5)

def _change_tempo(mid, factor):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        for msg in track:
            if not msg.is_meta:
                msg.time = int(msg.time * factor)
            new_track.append(msg)
        new_mid.tracks.append(new_track)
    return new_mid

def _gradual_tempo_change(mid, start_factor, end_factor):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        length = len(track)
        for i, msg in enumerate(track):
            if not msg.is_meta:
                progress = i / length
                factor = start_factor + (end_factor - start_factor) * progress
                msg.time = int(msg.time * factor)
            new_track.append(msg)
        new_mid.tracks.append(new_track)
    return new_mid

def note_played_too_soon(mid, offset=60):
    return _shift_note_timing(mid, -offset)

def note_played_too_late(mid, offset=60):
    return _shift_note_timing(mid, offset)

def _shift_note_timing(mid, offset):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        for msg in track:
            if not msg.is_meta and msg.type in ['note_on', 'note_off']:
                msg.time = max(0, msg.time + offset)
            new_track.append(msg)
        new_mid.tracks.append(new_track)
    return new_mid

def note_missing(mid, drop_rate=0.1):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        for msg in track:
            if msg.type == 'note_on' and random.random() < drop_rate:
                continue
            new_track.append(msg)
        new_mid.tracks.append(new_track)
    return new_mid

def note_not_expected(mid, insert_rate=0.05):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        for msg in track:
            new_track.append(msg)
            if msg.type == 'note_on' and random.random() < insert_rate:
                new_track.append(Message('note_on', note=random.randint(60, 72), velocity=64, time=0))
        new_mid.tracks.append(new_track)
    return new_mid

def articulated_legato(mid):
    return _modify_note_length(mid, stretch=1.5)

def articulated_staccato(mid):
    return _modify_note_length(mid, stretch=0.5)

def articulated_accentuated(mid):
    return _modify_velocity(mid, boost=40)

def _modify_note_length(mid, stretch):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        for msg in track:
            if msg.type == 'note_off':
                msg.time = int(msg.time * stretch)
            new_track.append(msg)
        new_mid.tracks.append(new_track)
    return new_mid

def _modify_velocity(mid, boost):
    new_mid = MidiFile()
    for track in mid.tracks:
        new_track = MidiTrack()
        for msg in track:
            if msg.type == 'note_on':
                msg.velocity = min(msg.velocity + boost, 127)
            new_track.append(msg)
        new_mid.tracks.append(new_track)
    return new_mid

def apply_modifications(input_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    mid = load_midi(input_path)

    mods = {
        'faster_tempo': faster_tempo,
        'slower_tempo': slower_tempo,
        'accelerando_tempo': accelerando_tempo,
        'ritardando_tempo': ritardando_tempo,
        'note_played_too_soon': note_played_too_soon,
        'note_played_too_late': note_played_too_late,
        'note_missing': note_missing,
        'note_not_expected': note_not_expected,
        'articulated_legato': articulated_legato,
        'articulated_staccato': articulated_staccato,
        'articulated_accentuated': articulated_accentuated
    }

    for name, func in mods.items():
        mod_mid = func(mid)
        save_midi(mod_mid, os.path.join(output_dir, f'{name}.mid'))


In [17]:
import os

# Carga el archivo MIDI original
input_path = 'midi/Acordai-guitarra.mid'
output_dir = 'modificaciones_midi'

os.makedirs(output_dir, exist_ok=True)

mid = load_midi(input_path)

# Aplica las modificaciones
modificaciones = {
    'faster_tempo': faster_tempo(mid,500),
    'slower_tempo': slower_tempo(mid,500),
    'accelerando_tempo': accelerando_tempo(mid),
    'ritardando_tempo': ritardando_tempo(mid),
    'note_played_too_soon': note_played_too_soon(mid),
    'note_played_too_late': note_played_too_late(mid),
    'note_missing': note_missing(mid),
    'note_not_expected': note_not_expected(mid),
    'articulated_legato': articulated_legato(mid),
    'articulated_staccato': articulated_staccato(mid),
    'articulated_accentuated': articulated_accentuated(mid)
}

# Guarda cada archivo modificado
for name, modified in modificaciones.items():
    output_path = os.path.join(output_dir, f"{name}.mid")
    save_midi(modified, output_path)
    print(f"Guardado: {output_path}")

Guardado: modificaciones_midi\faster_tempo.mid
Guardado: modificaciones_midi\slower_tempo.mid
Guardado: modificaciones_midi\accelerando_tempo.mid
Guardado: modificaciones_midi\ritardando_tempo.mid
Guardado: modificaciones_midi\note_played_too_soon.mid
Guardado: modificaciones_midi\note_played_too_late.mid
Guardado: modificaciones_midi\note_missing.mid
Guardado: modificaciones_midi\note_not_expected.mid
Guardado: modificaciones_midi\articulated_legato.mid
Guardado: modificaciones_midi\articulated_staccato.mid
Guardado: modificaciones_midi\articulated_accentuated.mid
